# Cipher Challenge
From [Simon Singh's The Code Book](https://simonsingh.net/cryptography/cipher-challenge/the-ciphertexts/)
## Stage 4
Vigenere Cipher

In [1]:
cipher_text = """    K Q O W E F V J P U J U U N U K G L M E K J I

    N M W U X F Q M K J B G W R L F N F G H U D W

    U U M B S V L P S N C M U E K Q C T E S W R E

    E K O Y S S I W C T U A X Y O T A P X P L W P

    N T C G O J B G F Q H T D W X I Z A Y G F F N

    S X C S E Y N C T S S P N T U J N Y T G G W Z

    G R W U U N E J U U Q E A P Y M E K Q H U I D

    U X F P G U Y T S M T F F S H N U O C Z G M R

    U W E Y T R G K M E E D C T V R E C F B D J Q

    C U S W V B P N L G O Y L S K M T E F V J J T

    W W M F M W P N M E M T M H R S P X F S S K F

    F S T N U O C Z G M D O E O Y E E K C P J R G

    P M U R S K H F R S E I U E V G O Y C W X I Z

    A Y G O S A A N Y D O E O Y J L W U N H A M E

    B F E L X Y V L W N O J N S I O F R W U C C E

    S W K V I D G M U C G O C R U W G N M A A F F

    V N S I U D E K Q H C E U C P F C M P V S U D

    G A V E M N Y M A M V L F M A O Y F N T Q C U

    A F V F J N X K L N E I W C W O D C C U L W R

    I F T W G M U S W O V M A T N Y B U H T C O C

    W F Y T N M G Y T Q M K B B N L G F B T W O J

    F T W G N T E J K N E E D C L D H W T V B U V

    G F B I J G Y Y I D G M V R D G M P L S W G J

    L A G O E E K J O F E K N Y N O L R I V R W V

    U H E I W U U R W G M U T J C D B N K G M B I

    D G M E E Y G U O T D G G Q E U J Y O T V G G

    B R U J Y S"""

Let's remove all the spaces and line breaks that appeared when copying and pasting from the web page.

In [2]:
cipher_text = "".join(cipher_text.split())

In [3]:
cipher_text

'KQOWEFVJPUJUUNUKGLMEKJINMWUXFQMKJBGWRLFNFGHUDWUUMBSVLPSNCMUEKQCTESWREEKOYSSIWCTUAXYOTAPXPLWPNTCGOJBGFQHTDWXIZAYGFFNSXCSEYNCTSSPNTUJNYTGGWZGRWUUNEJUUQEAPYMEKQHUIDUXFPGUYTSMTFFSHNUOCZGMRUWEYTRGKMEEDCTVRECFBDJQCUSWVBPNLGOYLSKMTEFVJJTWWMFMWPNMEMTMHRSPXFSSKFFSTNUOCZGMDOEOYEEKCPJRGPMURSKHFRSEIUEVGOYCWXIZAYGOSAANYDOEOYJLWUNHAMEBFELXYVLWNOJNSIOFRWUCCESWKVIDGMUCGOCRUWGNMAAFFVNSIUDEKQHCEUCPFCMPVSUDGAVEMNYMAMVLFMAOYFNTQCUAFVFJNXKLNEIWCWODCCULWRIFTWGMUSWOVMATNYBUHTCOCWFYTNMGYTQMKBBNLGFBTWOJFTWGNTEJKNEEDCLDHWTVBUVGFBIJGYYIDGMVRDGMPLSWGJLAGOEEKJOFEKNYNOLRIVRWVUHEIWUURWGMUTJCDBNKGMBIDGMEEYGUOTDGGQEUJYOTVGGBRUJYS'

## Strategy
To break the Vignere cipher, we need to find sequences of more than 4 letters that are repeated. Then we measure how many characters there are between the repetitions, so we can compute the length of the key. Then we can extract all the monoalphabetic ciphers, and work on each one.

## Finding the length of the key
### Figuring out repeated sequences



In [4]:
sequence_length = 10
total_length = len(cipher_text)
print(total_length)

604


In [5]:
word1 = cipher_text[:sequence_length]
print(word1)

KQOWEFVJPU


In [6]:
def check_length(sequence_length, cipher_text=cipher_text):
    """Check repetition of sequences of a given length in the cipher text"""
    repeated = []
    for i in range(1, total_length - 2 * sequence_length):
        seq_to_test = cipher_text[i:i+sequence_length]
        if (cipher_text.count(seq_to_test) > 1) & (seq_to_test not in repeated):
            repeated.append(seq_to_test)
    return repeated

In [41]:
repetition = {}
for seq_len in range(10, 3, -1):
    repetition[seq_len] = check_length(seq_len)

In [42]:
repetition

{10: [],
 9: [],
 8: [],
 7: ['WXIZAYG', 'NUOCZGM'],
 6: ['WXIZAY', 'XIZAYG', 'NUOCZG', 'UOCZGM'],
 5: ['WXIZA', 'XIZAY', 'IZAYG', 'NUOCZ', 'UOCZG', 'OCZGM', 'DOEOY'],
 4: ['EFVJ',
  'WXIZ',
  'XIZA',
  'IZAY',
  'ZAYG',
  'EKQH',
  'NUOC',
  'UOCZ',
  'OCZG',
  'CZGM',
  'EEDC',
  'DOEO',
  'OEOY',
  'IDGM',
  'FTWG',
  'WGMU']}

We notice that some shorter words are subparts of longer words, we should remove them.

In [43]:
all_words = [word for list_of_words in repetition.values() for word in list_of_words]

In [45]:
sub_words = [word for word1 in all_words for word in all_words if word in word1 and word != word1]

In [46]:
unique_words = set(all_words) - set(sub_words)

In [47]:
print(unique_words)

{'WXIZAYG', 'FTWG', 'EFVJ', 'DOEOY', 'EKQH', 'NUOCZGM', 'IDGM', 'WGMU', 'EEDC'}


Now we have the minimal list of repeated words, we have to find the distance between the repetition.

In [48]:
import re

In [52]:
indices = [[m.start() for m in re.finditer(word, cipher_text)]for word in unique_words]

In [53]:
indices

[[105, 295],
 [438, 483],
 [4, 224],
 [263, 308],
 [154, 374],
 [176, 256],
 [349, 514, 574],
 [440, 560],
 [193, 493]]

In [58]:
differences = []
for line in indices:
    iter_line = iter(line)
    start = next(iter_line)
    for i in iter_line:
        differences.append(i - start)
        start = i

In [59]:
differences

[190, 45, 220, 45, 220, 80, 165, 60, 120, 300]

In [62]:
from math import gcd
from functools import reduce

In [63]:
reduce(gcd, differences)

5

So there we have it, the length of the Vignere key is 5.